In [1]:
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 500)

In [3]:
import nibabel as nib
from nilearn.image import resample_img, new_img_like
from templateflow.api import get

In [4]:
import arviz as az

In [5]:
import bambi as bmb

In [6]:
def get_data_paths(suffix: str):
    if suffix in ["mask"]:
        pass
    elif suffix in ["z"]:
        suffix = f"stat-{suffix}_statmap"
    
    for subject in range(1, 17):
        yield Path("data") / (
            f"sub-{subject:02d}_task-faces_feature-taskBased_"
            "taskcontrast-facesGtScrambled_"
            "model-aggregateTaskBasedAcrossRuns_"
            f"contrast-intercept_{suffix}.nii.gz"
        )

In [7]:
template_path = get(template="MNI152NLin2009cAsym", resolution=2, desc="brain", suffix="T1w")
template_image = nib.load(template_path)

template_mask_path = get(template="MNI152NLin2009cAsym", resolution=2, desc="brain", suffix="mask")
template_mask_image = nib.load(template_mask_path)
template_mask_data = np.asanyarray(template_mask_image.dataobj, dtype=bool)

In [8]:
target_affine = template_image.affine
target_affine[:3,:3] *= 2.5
target_affine

array([[   5. ,    0. ,    0. ,  -96.5],
       [   0. ,    5. ,    0. , -132.5],
       [   0. ,    0. ,    5. ,  -78.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [9]:
mask = np.all(
    np.concatenate(
        [
            np.asanyarray(
                resample_img(
                    nib.Nifti1Image.from_filename(mask_path),
                    target_affine,
                    interpolation="nearest",
                ).dataobj
            ).astype(bool)[:, :, :, np.newaxis]
            for mask_path in get_data_paths("mask")
        ], 
        axis=3,
    ),
    axis=3,
)

In [10]:
x, y, z = np.nonzero(mask)
len(x)

14752

In [11]:
data_array = np.concatenate(
    [
        resample_img(
            nib.Nifti1Image.from_filename(zstat_path),
            target_affine,
        ).get_fdata()[x, y, z, np.newaxis]
        for zstat_path in get_data_paths("z")
    ], 
    axis=1,
)

In [12]:
data_frame = pd.DataFrame(data_array)
data_frame["voxel"] = np.ravel_multi_index((x, y, z), mask.shape)

data_frame = data_frame.melt(id_vars=["voxel"], var_name="subject")

In [13]:
data_frame

,voxel,subject,value
0,10095,0,-0.071804
1,10096,0,0.112519
2,10097,0,1.102691
3,10098,0,0.886668
4,10099,0,0.028903
...,...,...,...
236027,63060,15,0.801796
236028,63061,15,0.560480
236029,63062,15,0.135751
236030,63099,15,-0.280793


In [14]:
model = bmb.Model("value ~ (1|subject) + (1|voxel)", data_frame)

In [15]:
results = model.fit(
    tune=1000, 
    draws=1000, 
    chains=4, 
    method="nuts_numpyro",
    chain_method="parallel",
    nuts_kwargs=dict(max_tree_depth=10),
)

/fast/work/users/wallerl_c/miniforge3/envs/gpu/lib/python3.10/site-packages/aesara/link/jax/dispatch.py:87: UserWarning: JAX omnistaging couldn't be disabled: Disabling of omnistaging is no longer supported in JAX version 0.2.12 and higher: see https://github.com/google/jax/blob/main/design_notes/omnistaging.md.
  warnings.warn(f"JAX omnistaging couldn't be disabled: {e}")
/fast/work/users/wallerl_c/miniforge3/envs/gpu/lib/python3.10/site-packages/pymc/sampling_jax.py:36: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


Compiling...
Compilation time =  0:00:06.447796
Sampling...


/fast/work/users/wallerl_c/miniforge3/envs/gpu/lib/python3.10/site-packages/jax/linear_util.py:168: UserWarning: We will disable progress bar because it does not work yet on multi-GPUs platforms.
  ans = self.f(*args, **dict(self.params, **kwargs))


Sampling time =  0:23:11.094515
Transforming variables...
Transformation time =  0:00:01.099567


In [16]:
results.to_netcdf("results.nc")

'results.nc'

In [17]:
number_of_divergences = int(results.sample_stats.diverging.sum())
number_of_divergences

0

In [19]:
bool(np.all((0.9 <= az.rhat(results) <= 1.05).to_array()))

True

In [20]:
effect = results.posterior["1|voxel"]

In [21]:
x, y, z = np.unravel_index(list(map(int, effect.voxel__factor_dim)), mask.shape)

In [24]:
posterior_map = np.zeros(template_image.shape)
posterior_map[x, y, z] = effect.mean(axis=(0, 1))

In [25]:
posterior_map_image = resample_img(
    nib.Nifti1Image(posterior_map, target_affine),
    template_image.affine,
    interpolation="nearest",
)

In [26]:
nib.save(posterior_map_image, "posterior.nii.gz")

In [30]:
probability_map = np.zeros(template_image.shape)
probability_map[x, y, z] = ((effect > 0).mean(axis=(0, 1)) - 0.5) * 2

probability_map_image = resample_img(
    nib.Nifti1Image(probability_map, target_affine),
    template_image.affine,
    interpolation="nearest",
)

nib.save(probability_map_image, "probability.nii.gz")